In [3]:
## Importing Libraries 


import numpy as np
import pandas as pd
import os

import seaborn as sns
import matplotlib.pyplot as plt


from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import KFold


# Deep Learning Imports
import tensorflow as tf
import keras
from keras.models import Sequential, load_model, Model
from keras.layers import Dense, Conv1D, Activation, Dropout, Flatten, MaxPooling1D, Input 
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.activations import tanh
from keras.optimizers import Adam
from keras.utils.np_utils import to_categorical # convert to one-hot-encoding


In [ ]:
## For GPU

import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

## Load Datasets

In [ ]:
from google.colab import files # to load dataset on google colab

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "Uploading_Data_Colab_1.xlsx" with length 9000 bytes'.format(
      name=fn, length=len(uploaded[fn])))

In [9]:
# Dataset under 1500rpm motor speed and 10Nm load using X direction
x_1500 = np.load('x_1500_10.npy')
y_1500 = np.load('gt_1500_10.npy') 
x_1500_test = np.load('x_test_1500_10.npy')


# Dataset under 2700rpm motor speed and 25Nm load using X direction
x_2700 = np.load('x_2700_25.npy')
y_2700 = np.load('gt_2700_25.npy')
x_2700_test = np.load('x_test_2700_25.npy')

## Data Preprocessing

In [10]:
# Define the to_3d function to change dataset to format suitable for keras model

def to_3d(data):
    data = np.reshape(data, data.shape+ (1,))
    return data


In [11]:
X_1500 = to_3d(x_1500)
X_1500_test = to_3d(x_1500_test)

X_2700 = to_3d(x_2700)
X_2700_test = to_3d(x_2700_test)

## Build 1D CNN Model for Vibration Signal feature extraction

In [12]:
def model():
     
    num_classes = len(np.unique(y_2700))
    input_size = X_2700.shape

    conv1 = Sequential()
    conv1.add(Conv1D(128,100, activation='relu', padding= 'same', input_shape = input_size[1:]))
    conv1.add(MaxPooling1D(pool_size=2))
    conv1.add(Dropout(rate=0.3))

    conv1.add(Conv1D(64, 50, activation='relu', padding= 'same'))
    conv1.add(MaxPooling1D(pool_size=2))
    conv1.add(Dropout(rate=0.3))

    conv1.add(Conv1D(32, 25, activation='relu', padding= 'same'))
    conv1.add(MaxPooling1D(pool_size=2)) 
    conv1.add(Dropout(rate=0.25))

    conv1.add(Conv1D(16, 25, activation='relu', padding= 'same'))
    conv1.add(MaxPooling1D(pool_size=2))

    conv1.add(Conv1D(8, 25, activation='relu', padding= 'same'))
    conv1.add(Dropout(rate=0.25))

    conv1.add(Flatten())
    conv1.add(Dense(100, activation='relu'))
    conv1.add(Dense(num_classes, activation='softmax'))


    conv1.compile(optimizer = Adam(learning_rate = 0.001), loss='categorical_crossentropy', metrics=['accuracy'])

    return conv1



## Model Summary

In [ ]:
model().summary()

## Train Model

In [14]:
def train_model(x_data, y_data, test_data): 

    n_split = 5
    cv_scores = [] # create a list of cross-validation (cv) scores for all cv sets
    cms = [] # create a list of confusion matrices for all cv sets
    history = [] # create a list of model history for all cv sets
    test_data_pred = [] # create list of model prediction on test data for all cv sets

    with tf.device('/device:GPU:0'): # run with google colab GPU
    
        for train_index,test_index in KFold(n_split).split(x_data):

          x_train,x_test= x_data[train_index], x_data[test_index]
          y_train,y_test= y_data[train_index], y_data[test_index]
          Y_train = to_categorical(y_train, num_classes = 5)
          Y_test = to_categorical(y_test, num_classes = 5)


          batch_size= 200
          num_epochs= 25
  
          cnn_model= model()

          hist = cnn_model.fit(x_train, Y_train,epochs=num_epochs, batch_size=batch_size, verbose=1,validation_data=(x_test,Y_test),shuffle=True)
          history.append(hist)


          # predict model
          y_pred = cnn_model.predict(x_test)

          y_pred_classes = np.argmax(y_pred,axis = 1) # Convert predictions classes to one hot vectors  
          cm = confusion_matrix(y_test, y_pred_classes,normalize='true') # results from confusion matrix 
          cms.append(cm)

          # model prediction on test data
          test_data_pred.append(cnn_model.predict(test_data))


          #evaluate model
          scores = cnn_model.evaluate(x_test,Y_test, verbose=0)
          print("%s: %.2f%%"% (cnn_model.metrics_names[1], scores[1]*100))
          cv_scores.append(scores[1]*100)
    
        print("%.2f%% (+/- %.2f%%)"%(np.mean(cv_scores), np.std(cv_scores)))

    return cv_scores, cms, history, test_data_pred
      
    


In [ ]:
## Train Model for 1500_10 condition dataset

cv_scores, cms, history, test_data_predictions = train_model(X_1500, y_1500, X_1500_test)

In [ ]:
## Train Model for 2700_25 condition dataset

cv_scores_27, cms_27, history_27, test_data_predictions_27 = train_model(X_2700, y_2700, X_2700_test)

## Get Model Predictions on Test Data

In [24]:
def predict(test_predictions, cv_scores):
    
    k = np.argmax(cv_scores)
    return np.argmax(test_predictions[k],axis = 1) # Convert to classes 

In [87]:
##  saving predictions on 1500_10 operation condition dataset 

from numpy import savetxt

savetxt('predictions.csv', predict(test_data_predictions, cv_scores=cv_scores))




In [25]:
## saving predictions on 2700_25 operation condition dataset 

from numpy import savetxt

savetxt('predictions_27.csv', predict(test_data_predictions_27, cv_scores=cv_scores_27))

## Model Performance

In [18]:
def model_performance(model_history, cv_scores, cms):
    
    k = np.argmax(cv_scores) # index of maximum cvscore
    cm = cms[k]
    accuracy = cv_scores[k]
    print("Model Accuracy = %.2f%% \n"%(accuracy) )

    # Plotting confusion matrix

    f = sns.heatmap(cm, annot=True,xticklabels= ['normal', 'surface \n wear', 'crack' , 'chipped', 'tooth \n missing'], yticklabels= ['normal', 'surface wear', 'crack' , 'chipped', 'tooth missing'], cmap="Greens")
    plot = plt.show()

    return plot

In [ ]:
## Model performance for 1500_10 condition dataset
model_performance(model_history=history, cv_scores = cv_scores, cms= cms)

In [ ]:
## Model performance for 2700_25 condition dataset
model_performance(model_history=history_27, cv_scores = cv_scores_27, cms= cms_27)